In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [2]:
data = pd.read_csv("Churn_Modelling.csv")
data.head()
len(data)

10000

In [3]:
## Preprocess the data
### Drop irrelevant columns
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [4]:
## Encode categorical variables
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## Onehot encode 'Geography
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoder

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [6]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [8]:
## Combine one hot encoder columns with the original data
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [9]:
# is saving your trained LabelEncoder object to a file so that you can reuse the exact same encoder later
# Pickle serializes the encoder object (LabelEncoder) and saves it to a file.
# It freezes the encoder in its current state (classes, mapping, configuration).
# Because your model was trained on encoded data. If this mapping changes even slightly, predictions become meaningless.
# You ensure data preprocessing is identical between training and deployment.
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender, file)

In [10]:
with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(onehot_encoder_geo, file)

In [11]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
## DiVide the dataset into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

## Split the data in training and tetsing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

## Scale these features
# StandardScaler is a tool from scikit-learn that normalizes numerical features.
# Why NOT use fit_transform on X_test?
# Because test data must never influence the model.
# If u do you, you leak information from the test set into training → data leakage → fake high accuracy.
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [14]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [14]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [15]:
data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


### ANN Implementation

In [16]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [17]:
X_train.shape

(8000, 12)

In [18]:
(X_train.shape[1],)

(12,)

In [19]:
## Build Our ANN Model 
## create a Sequential Network and use Dense to create neuron and use Activation function (relu, sigmoid)
model=Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32, activation='relu'), ## HL2
    Dense(1, activation='sigmoid') ## output layer
])

2025-11-15 13:31:32.580435: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M4
2025-11-15 13:31:32.580483: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-11-15 13:31:32.580491: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.92 GB
2025-11-15 13:31:32.580571: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-15 13:31:32.580599: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [21]:
import tensorflow
opt=tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy()
loss

In [22]:
## use Optimizer useful for Back Propogation for updating weights
## loss function
## Metrics >> In Classification Probelm we have ["accuracy"] and for regression problem we have ["msc", "mae"] Mean Squared Error, Mean Average Error
model.compile(optimizer=opt,loss="binary_crossentropy",metrics=['accuracy'])

In [23]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [24]:
## Set up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [25]:
### Train the model
# One complete pass of the entire training dataset through the model.
# If you have 1000 training samples, then:
# 1 epoch = the model sees all 1000 samples once
# 5 epochs = the model sees all samples 5 times
# if after a particular epocs our loss value is not decreasing we apply early stoping to stop the training before the specified epochs value
# be patience for atleast 5 epochs and restore the best weight u find
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
 16/250 [>.............................] - ETA: 0s - loss: 0.5571 - accuracy: 0.7168  

2025-11-15 13:31:44.987184: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
2025-11-15 13:31:45.002637: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node Adam/AssignAddVariableOp.


250/250 [==============================] - 2s 5ms/step - loss: 0.6881 - accuracy: 0.7617 - val_loss: 0.6999 - val_accuracy: 0.7710
Epoch 2/100
250/250 [==============================] - 1s 4ms/step - loss: 2.4211 - accuracy: 0.7138 - val_loss: 7.9774 - val_accuracy: 0.6090
Epoch 3/100
250/250 [==============================] - 1s 4ms/step - loss: 5.6363 - accuracy: 0.7264 - val_loss: 7.3353 - val_accuracy: 0.6300
Epoch 4/100
250/250 [==============================] - 1s 4ms/step - loss: 10.0409 - accuracy: 0.7184 - val_loss: 11.4079 - val_accuracy: 0.7075
Epoch 5/100
250/250 [==============================] - 1s 4ms/step - loss: 13.8133 - accuracy: 0.7207 - val_loss: 8.5631 - val_accuracy: 0.7380
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 21.6637 - accuracy: 0.7264 - val_loss: 19.4893 - val_accuracy: 0.6630
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 23.8787 - accuracy: 0.7234 - val_loss: 32.6185 - val_accuracy: 0.6510
Epoc

In [26]:
# model.save('model.h5')
model.save('model.h5')

/opt/anaconda3/envs/py310/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
## Load Tensorboard Extension
%load_ext tensorboard

In [28]:
# (py310) shashankshukla@Shashanks-MacBook-Pro ANN % tensorboard --logdir logs/fit
# TensorBoard 2.15.2 at http://localhost:6007/ (Press CTRL+C to quit)
%tensorboard --logdir logs/fit